# Install Depencies

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
! pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00


In [2]:
! pip install transformers
! pip install sentencepiece
# ! pip install datasets
! pip install transformers[torch]
! pip install peft
! pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00


In [3]:
#
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [4]:
 !pip install transformers pandas datasets accelerate sentencepiece tokenizers peft


# Question answering Model

In [5]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM ,pipeline


model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
# ,torch_dtype=torch.bfloat16

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
# فكشن تطبعلك عدد الترينبل باراميترز
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 354823168
all model parameters: 354823168
percentage of trainable model parameters: 100.00%


In [ ]:
# Let's predict the next word for a given sentence in old model.
sentence = 'i have haigh blood'
device = "cpu"

tokens = tokenizer(sentence, return_tensors='pt').to(device)

# Get the prediction
prediction = model.generate(**tokens)

# Get the id of the predicted token

predicted_id = prediction.argmax(-1).item()

# Decode the token
predicted_token = tokenizer.decode([predicted_id])

print(f'The predicted next word is: {predicted_token}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


The predicted next word is: &


In [ ]:
# fine tune

from datasets import load_dataset
#  حملت البيانات
dataset = load_dataset("health360/Healix-V1")

Generating train split:   0%|          | 0/796239 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
#  هنا ما اخذ كل البيانات
# اخذ فقط عشره بالميه منها
# Calculate the number of examples
num_examples = len(dataset['train'])

num_small = num_examples//20

# Generate random indices
# و استعمل الراندوم جويس حته يختار انديكسات تكون من الداتا الجبيره
# بس عددها ما يتخطى العشره بالميه
indices = np.random.choice(num_examples, num_small, replace=False)

# Select the randomly chosen examples
smaller_dataset = dataset['train'].select(indices)
# smaller_dataset = dataset['train']

In [ ]:
import re
#  و هنا بس نضف التيكست
def clean_text(example):
  text = example['text'].replace("### Input:", '')
  text = text.replace("### Response:", '')
  text = text.replace("\n", '')
  return {'text': text}

In [ ]:
smaller_dataset = smaller_dataset.map(clean_text)

Map:   0%|          | 0/39811 [00:00<?, ? examples/s]

In [ ]:
# 796239 full data
# 79623 -10%
# 159247 -5%

In [ ]:
len(smaller_dataset)

39811

In [ ]:
smaller_dataset[0]

{'text': 'how are bioengineered skin substitutes categorized?- bioengineered skin substitutes can be categorized based on the composition of the source material (synthetic versus biologic, human versus animal), intended duration (permanent versus temporary), intended tissue to be replaced (epidermal versus dermal), and number of layers (single versus bilaminate).'}

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # Tokenize the examples
    output = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

    # Make sure the labels are the same as the input ids
    output["labels"] = output["input_ids"].copy()

    return output


# Tokenize the dataset
tokenized_dataset = smaller_dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/39811 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 39811
})

In [ ]:
#num_rows: 398119

In [ ]:
# spliting
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
train_data, val_data = train_test_split(tokenized_dataset, test_size=0.2, random_state=42)

In [ ]:
# Now, you can create separate datasets for training and validation
train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)

In [ ]:
train_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 31848
})

In [ ]:
val_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 7963
})

In [ ]:
from transformers import Trainer, TrainingArguments

# Define the training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Graduation_Models/Question_Answering_Model/res",
    num_train_epochs=5,
    per_device_train_batch_size=1,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/Graduation_Models/Question_Answering_Model/log",
    learning_rate=2e-5,

     evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,

    fp16=True,
    logging_steps=100,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
)

In [ ]:
# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    )

In [ ]:
# Start training
# resume_from_checkpoint=True
trainer.train(resume_from_checkpoint=True)
# start 1:23 - 4 first epoch

There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
5,1.586400,1.468190


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=199055, training_loss=0.3090007614009643, metrics={'train_runtime': 9603.5306, 'train_samples_per_second': 20.727, 'train_steps_per_second': 20.727, 'total_flos': 4.621562918731776e+16, 'train_loss': 0.3090007614009643, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.5388340950012207,
 'eval_runtime': 124.9435,
 'eval_samples_per_second': 63.733,
 'eval_steps_per_second': 7.972,
 'epoch': 5.0}

In [9]:
# Testing
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model_ = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/Graduation_Models/Question_Answering_Model/res/checkpoint-199055")

In [10]:
# فكشن تطبعلك عدد الترينبل باراميترز
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model_))

trainable model parameters: 354823168
all model parameters: 354823168
percentage of trainable model parameters: 100.00%


In [ ]:
prompt = "What are the symptoms of pneumonia?"
device = "cpu"

# Tokenize the input prompt
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

# Define generation parameters
max_length = 100
num_return_sequences = 1
top_k = 50  # Adjust this parameter for top-k sampling if needed
top_p = 0.9  # Adjust this parameter for nucleus sampling if needed

# Generate the output answer
output = model.generate(
    input_ids,
    max_length=max_length,
    num_return_sequences=num_return_sequences,
    top_k=top_k,
    top_p=top_p,
    temperature=1.0  # Adjust temperature if needed
)

# Decode the generated answer
generated_answer = tokenizer.decode(output[0], skip_special_tokens=True)

print(f'The generated answer is: {generated_answer}')


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The generated answer is: What are the symptoms of pneumonia?symptoms of pneumonia can include fever, cough, shortness of breath, chest pain, and fatigue.


# Next word prediction

In [ ]:
import pandas as pd

In [ ]:
data_ = pd.read_csv('/content/drive/MyDrive/all_data.csv',index_col=0)

In [ ]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
data_[104:105]

,LLM Output
104,"A patient with a history of 3-5 years and multigenic tests showed Luminal B-like characteristics, including HER2 negativity, ER positivity, and Endocrine therapy. However, for those with HER2 overexpression, Cytotoxic Oxics and anti-HER2 therapy are recommended. In cases with Ki-67 ≥ 20-29%, or PgR low/negative, and high recurrence risk, Cytotoxic Oxics or most therapies are suitable. Patients with Luminal B-like features and HER2 overexpression, Cytotoxic Oxics and anti-HER2 therapy are recommended, particularly for those with HER2 amplification. In patients with negative ER and PgR, Cytotoxic Oxics with 80% overlap between Triple-negative subtypes are suitable. A minimum of 3-5 years of more than 20% positivity is recommended in some cases."


In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset

In [11]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM ,pipeline


model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [12]:
# فكشن تطبعلك عدد الترينبل باراميترز
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 354823168
all model parameters: 354823168
percentage of trainable model parameters: 100.00%


In [ ]:
import pandas as pd


# Determine the number of rows to select for 20% sample
sample_size = int(len(data_) * 0.6)

# Randomly select rows indices
sampled_indices = data_.sample(n=sample_size, random_state=42).index

# Get the corresponding rows from the original DataFrame
data_sampled = data_.loc[sampled_indices]

len(data_sampled)

20023

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
data_sampled[100:105]

,LLM Output
29945,"In studies with censored data, linear survival curves are commonly used to display percentage survival (or freedom from a complication) on the x-axis and time on the y-axis. The curve drops as each study patient dies or gets the complication, and if there are few patients, it may be possible to see individual deaths."
11836,"MRI reveals characteristics of most regulating spatial expression of proteins involved in cartilage in ous tumors ors: intermediate to low signal intensity on T1-weighted and weighted ed and high signal intensity on T2- also expressed in chondroma fibroma."" \n\nNote: This sentence is extracted from the given text, which contains medical information related to MRI characteristics of proteins in cartilage-related tumors ors. It is suitable for fine-tuning a medical model"
31161,"Morbid jealousy, commonly associated with alcohol abuse, is a major contributor to the frequency of wife battering and is present in 35% of men and 31% of women who abuse a commonest motivator for homicide, according to studies conducted in 1982 and 1990. Delusional jealousy, often accompanied by violence, is more often vented on the partner than on the supposed rival, especially in cases of alcohol abuse, and is a common factor in homicide data from Detroit in 1972. In some cases, morbid homicide data suggests that jealousy is justified, but ultimately motivated by sexual jealousy."
9048,"n most cases of relapsed ALL, there is a clear clonal relationship between the diagnostic karyotype and that of the original karyotype at the time of diagnosis, with approximately 94% of patients with pre-B ALL and 71% of patients with T-cell ALL exhibiting such relationships. By analyzing patterns of genomic copy number abnormalities, it is clear that CNAs were either present at low levels at diagnosis and then selected for at relapse or newly acquired on the basis of new genomic alterations after initial induction therapy. Only a minority of cases have no change in their genomic copy number abnormalities, with the rest exhibiting either a secondary leukemia emic clone or a leukemia emia that arose from an initial leukemia"
566,study has shown that patients with one receptor or high-grade tumors have been associated with a particular genetic signature that predicts improved response to neoadjuvant chemotherapy. The incorporation of genomic analyses in the clinical setting allows for the identification of molecular predictors of response to a given treatment and provides insight into the underlying mechanisms of response


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(text):
    # Convert the text to a string if it's not already
    text = str(text)
    # Tokenize the text
    output = tokenizer(text, padding="max_length", truncation=True, max_length=128)

    # Make sure the labels are the same as the input ids
    output["labels"] = output["input_ids"].copy()

    return output

# Extract the text column from your DataFrame
texts = data_sampled["LLM Output"].tolist()

# Tokenize the text using the custom function
tokenized_data = [tokenize_function(text) for text in texts]

# Combine the tokenized data into a single Dataset
tokenized_dataset_1 = Dataset.from_dict({key: [item[key] for item in tokenized_data] for key in tokenized_data[0]})

# Print the structure of the resulting dataset
print(type(tokenized_dataset_1))
print(tokenized_dataset_1)


<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 20023
})


In [ ]:
# spliting
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
train_data, val_data = train_test_split(tokenized_dataset_1, test_size=0.2, random_state=42)

In [ ]:
# Now, you can create separate datasets for training and validation
train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)

In [ ]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 16018
})

In [ ]:
val_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 4005
})

In [ ]:
from transformers import Trainer, TrainingArguments

# Define the training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Graduation_Models/Next_word_model/output",
    num_train_epochs=5,
    per_device_train_batch_size=1,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/Graduation_Models/Next_word_model/Log",
    learning_rate=2e-5,
    fp16=True,
    logging_steps=100,

    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,

    optim="adamw_torch",
    lr_scheduler_type="cosine",
)



In [ ]:

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    )

In [ ]:
# Start training
# resume_from_checkpoint=True
trainer.train(resume_from_checkpoint=True)
# start 2:30 - 3:30

There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=80090, training_loss=0.0, metrics={'train_runtime': 17.7557, 'train_samples_per_second': 4510.673, 'train_steps_per_second': 4510.673, 'total_flos': 1.859490965618688e+16, 'train_loss': 0.0, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 2.053778886795044,
 'eval_runtime': 51.698,
 'eval_samples_per_second': 77.469,
 'eval_steps_per_second': 9.691,
 'epoch': 5.0}

In [14]:
# Testing
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model__ = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/Graduation_Models/Next_word_model/output/checkpoint-80090")

In [15]:
# فكشن تطبعلك عدد الترينبل باراميترز
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model__))

trainable model parameters: 354823168
all model parameters: 354823168
percentage of trainable model parameters: 100.00%


In [ ]:
# Let's predict the next word for a given sentence in old model.
# sentence = "The patient was diagnosed with a rare autoimmune disease characterized by atypical"
# sentence = "In the patient's medical chart, the doctor noted a history of chronic"
# sentence = "Patient's symptoms require additional tests for conclusive"
sentence = 'i have willnes'
device = "cpu"
tokens = tokenizer(sentence, return_tensors='pt').to(device)

# Get the prediction
prediction = model(**tokens )

# Get the id of the predicted token
predicted_id = prediction.logits[...,-1,:].argmax(-1).item()

# Decode the token
predicted_token = tokenizer.decode([predicted_id])

print(f'The predicted next word is: {predicted_token}')


The predicted next word is:  disease


# Summarazation model

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer


In [ ]:
model_name='google/flan-t5-base'
# هنا حملت المودل بس لاحظ استعملت خاصيه ال quantization
#  و استعملت الbfloat16 حته اقلل حجم المودل للنص
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
tokenizer.save_pretrained('com')

('com/tokenizer_config.json',
 'com/special_tokens_map.json',
 'com/spiece.model',
 'com/added_tokens.json',
 'com/tokenizer.json')

In [ ]:
# فكشن تطبعلك عدد الترينبل باراميترز
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [ ]:
from datasets import load_dataset
import time


In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [ ]:
# نستعمل الزيرو شوت بحيث مننطي للمودل اي مساعده مجرد جمله و نكله لخص


# اختار انديكس عشوائي
index = 200
# اختار نص و التلخيص مالته
dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

-------------------------------------------------------------------

In [ ]:
# هاي الفكشن هي تسوي البيانات بالشكل المطلوب حيث بيها الجمله او التعلميات للمودل و ايضا بعدها تخلي
# الدايلوك اوالجمله الي اريد الخصها و بعدها كلمه summary
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    # و هنا خلا بيها الانبوت ايديس و ليبلز
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
# اول شي الترين و التيست و الفال هذني موجودات بالداتا الاصليه ما لعبت بيهن
# و ايضا بدلت او مسحت الاعمده القديمه و خليت الجديده الي هي الانبوت ايدس و ليبلز
# و طبعا الفكشن سوت لكلشي توكينايز و كلشي تمام للمودل
tokenized_datasets


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1500
    })
})

In [ ]:
# هنا اخذ جزء من الداتا مو كلها علمود الوقت
# tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)
tokenized_datasets = tokenized_datasets

In [ ]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

# print(tokenized_datasets)

Shapes of the datasets:
Training: (12460, 2)
Validation: (500, 2)
Test: (1500, 2)


In [ ]:
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments


In [ ]:
# هسه اجيت اسوي فاين تيون ف طريقه التدريب ب هاكنك فيس ك التالي تاخذ ترينر و تريننك اركيومنت


# هنا الاوت بوت اي المودل وين يتخزن
output_dir = f'/content/drive/MyDrive/Graduation_Models/Summary_Model/output'
logging_dir = f'/content/drive/MyDrive/Graduation_Models/Summary_Model/Log'




# Define the training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir=logging_dir,
    learning_rate=2e-5,
    logging_steps=100,
    save_strategy ='epoch',
    optim="adamw_torch",
    lr_scheduler_type="cosine",
)




def compute_metrics(p):
    # Customize this function based on your evaluation needs
    return {"perplexity": p.metrics["eval_loss"]}

# Create a data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set to True if you are using a masked language model
)

# هنا اجيب انطيته بيانات التدريب و الاختبار و المودل و الاركيومنت
trainer = Trainer(
    model=original_model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],

)

In [ ]:
# هسه نبدي ندرب
#ساعه  و نص للايبوك الواحد
trainer.train(resume_from_checkpoint=True)

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


Step,Training Loss
31600,0.033000
31700,0.048800
31800,0.036600
31900,0.039900
32000,0.030400
32100,0.031900
32200,0.033400
32300,0.041000
32400,0.030500
32500,0.038400


In [ ]:
import os

In [ ]:
checkpoint_path = "/content/drive/MyDrive/Graduation_Models/Summary_Model/output/checkpoint-30000"

if not os.path.exists(checkpoint_path):
    raise ValueError(f"Checkpoint not found at {checkpoint_path}")

In [ ]:
trainer.train(resume_from_checkpoint=checkpoint_path)


KeyboardInterrupt: 

In [ ]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Graduation_Models/Summary_Model/output/checkpoint-12460")

In [ ]:
original_model = original_model.to('cuda')
instruct_model = instruct_model.to('cuda')


In [ ]:
# هنا اختبرت المودل و اقدرته ورا الفاين تيون
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize this conversation.

{dialogue}

Summary:
"""
top_p = 0.5
top_k = 3

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to('cuda')


original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1,))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
#Person1#: I'm thinking of upgrading my computer.
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
#Person1#: Have you considered upgrading your system? #Person2#: Yes, but I'm not sure what exactly I would need. #Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising. #Person2#: That would be a definite bonus. #Person1#: You might also want to upgrade your hardware because it is pretty outdated now. #Person2#: How can we do that? #Person1#: You'd probably need a faster processor, more memory and a faster mo

In [ ]:
len(human_baseline_summary)

87

In [ ]:
len(original_model_text_output)

49

In [ ]:
len(instruct_model_text_output)

692

In [ ]:
import evaluate


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=ca8c4fa0777842dc3dfda36233de5f622e87bc7c0d89d02d69aa5e7abe94f4d3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
# Evaluate the Model Quantitatively (with ROUGE Metric)

rouge = evaluate.load('rouge')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# هنا اخذ بس عشر سامبلات و قيم بيهن المودل الاصلي بدون تعديل و قيم ايضا المودل الي سويتله فاين تيون
# و ب روج طبعا

dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []

for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,[ purpose ] this memo should go out as an intr...,[ purpose ] [ purpose ] [ subject to this new ...
1,In order to prevent employees from wasting tim...,[ purpose ] this memo should go out as an intr...,[ purpose ] [ purpose ] [ subject to this new ...
2,Ms. Dawson takes a dictation for #Person1# abo...,[ purpose ] this memo should go out as an intr...,[ purpose ] [ purpose ] [ subject to this new ...
3,#Person2# arrives late because of traffic jam....,a lot of traffic jams are causing pollution in...,#Person1#: You're finally here! What took so l...
4,#Person2# decides to follow #Person1#'s sugges...,a lot of traffic jams are causing pollution in...,#Person1#: You're finally here! What took so l...
5,#Person2# complains to #Person1# about the tra...,a lot of traffic jams are causing pollution in...,#Person1#: You're finally here! What took so l...
6,#Person1# tells Kate that Masha and Hero get d...,Summary: a couple of girls who have a separati...,Summary: Kate and Hero are getting divorced. K...
7,#Person1# tells Kate that Masha and Hero are g...,Summary: a couple of girls who have a separati...,Summary: Kate and Hero are getting divorced. K...
8,#Person1# and Kate talk about the divorce betw...,Summary: a couple of girls who have a separati...,Summary: Kate and Hero are getting divorced. K...
9,#Person1# and Brian are at the birthday party ...,"a great party. #Person1#: Happy Birthday, this...","Brian, Brian, may I have a pleasure to have a ..."


In [ ]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.14495877038053528, 'rouge2': 0.025132885690161234, 'rougeL': 0.10734854191439931, 'rougeLsum': 0.10598143808643898}
INSTRUCT MODEL:
{'rouge1': 0.16403334487739019, 'rouge2': 0.05360793615254001, 'rougeL': 0.13371414071699528, 'rougeLsum': 0.13496962262442147}


In [ ]:
print("Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL")
# نسبه مئويه لمقدار التقدم عن المودل العادي بدون فاين تيون
improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL
rouge1: 1.91%
rouge2: 2.85%
rougeL: 2.64%
rougeLsum: 2.90%


In [ ]:
# peft
# هسه حنبدي نشتغل ب لورا و الي تجمد الويتس مال مودل و تجيب ابعاد اصغرr
# و تدربهم ع بيانات الفاين تيون و بعدها ترجع و تدمجهن بالمودل الجديد

from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    # و هذا الدايمنشن الي كلنه يكون بين ال8 و 32
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    # هذا نوع التاسك
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [ ]:
# هنا انطيته المودل مالتي حته يجمده و انطيته اعدادات لورا حته يضبطها
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


In [ ]:
# هسه اجيت اسوي فاين تيون ف طريقه التدريب ب هاكنك فيس ك التالي تاخذ ترينر و تريننك اركيومنت


# هنا الاوت بوت اي المودل وين يتخزن
output_dir = f'/content/drive/MyDrive/Graduation_Models/Summary_Model/peftmodel/output'
logging_dir = f'/content/drive/MyDrive/Graduation_Models/Summary_Model/peftmodel/log'


# Define the training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir=logging_dir,
    learning_rate=2e-5,
    save_strategy ='epoch',
    logging_steps=100,

    optim="adamw_torch",
    lr_scheduler_type="cosine",
)




# # و هنا الاركيومنت مال التدريب
# training_args = TrainingArguments(
#     output_dir=output_dir,
#     learning_rate=1e-5,
#     # ايبوك واحد هنا
#     num_train_epochs=4,
#     warmup_steps=100,
#     weight_decay=0.01,


#     logging_steps=100,
#     max_steps=1
# )


# هنا اجيب انطيته بيانات التدريب و الاختبار و المودل و الاركيومنت
peft_trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)









In [ ]:
peft_trainer.train()

Step,Training Loss


KeyboardInterrupt: 

In [ ]:
# هنا حتدمج المودل مالتك ويا المودل مال لورا

from peft import PeftModel, PeftConfig

# هنا حملت المودل الاصلي بدون اي تعديل و التوكينايزر مالته
peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("Falconsai/medical_summarization", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("Falconsai/medical_summarization")

# رجعت هنا انطيته المودل الاصلي و بعدها رابط المودل مال لورا او الرابط مالته و ايضا سويت كوانتيزيشن
# و اخر باراميتر خليته فولز لان بعد ما اريد اسوي عليه فاين تيون يعني اريد هسه اسويله ديبلوي
peft_model = PeftModel.from_pretrained(peft_model_base,
                                       '/content/Peft/checkpoint-500',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)
# بالتالي الترنبل باراميتر يصير 0

In [ ]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 0
all model parameters: 61686272
percentage of trainable model parameters: 0.00%


In [ ]:
# تقييم حسب الانسان
index = 200
dialogue = dataset['test'][index]['dialogue']
baseline_human_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
Summary: a new software program is coming out on Cds.results:the following conversation is discussed.results:the following conversation is a discussion.results:the following conversation is a discussion.results:the following conversation was a discussion.results:the following conversation was a discussion.results:the following conversation was a discussion.results:the following conversation was a discussion.conclusions:the following conversation is a few.
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
#Person1#: Have you considered upgrading your system? #Person1#: Yes, but I'm not sure what exac

In [ ]:
# و هنا حيقيم ب روج لل3 مودلات

dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    human_baseline_text_output = human_baseline_summaries[idx]

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,[ purpose ] this memo should go out as an intr...,[ purpose ] [ purpose ] [ subject to this new ...,[ purpose ] this memo should go out as an intr...
1,In order to prevent employees from wasting tim...,[ purpose ] this memo should go out as an intr...,[ purpose ] [ purpose ] [ subject to this new ...,[ purpose ] this memo should go out as an intr...
2,Ms. Dawson takes a dictation for #Person1# abo...,[ purpose ] this memo should go out as an intr...,[ purpose ] [ purpose ] [ subject to this new ...,[ purpose ] this memo should go out as an intr...
3,#Person2# arrives late because of traffic jam....,a lot of traffic jams are causing pollution in...,#Person1#: You're finally here! What took so l...,Summary: The following conversation was conduc...
4,#Person2# decides to follow #Person1#'s sugges...,a lot of traffic jams are causing pollution in...,#Person1#: You're finally here! What took so l...,Summary: The following conversation was conduc...
5,#Person2# complains to #Person1# about the tra...,a lot of traffic jams are causing pollution in...,#Person1#: You're finally here! What took so l...,Summary: The following conversation was conduc...
6,#Person1# tells Kate that Masha and Hero get d...,Summary: a couple of girls who have a separati...,Summary: Kate and Hero are getting divorced. K...,Summary: a short summary of the discussion is ...
7,#Person1# tells Kate that Masha and Hero are g...,Summary: a couple of girls who have a separati...,Summary: Kate and Hero are getting divorced. K...,Summary: a short summary of the discussion is ...
8,#Person1# and Kate talk about the divorce betw...,Summary: a couple of girls who have a separati...,Summary: Kate and Hero are getting divorced. K...,Summary: a short summary of the discussion is ...
9,#Person1# and Brian are at the birthday party ...,"a great party. #Person1#: Happy Birthday, this...","Brian, Brian, may I have a pleasure to have a ...",a great party. a great party. a great party. a...


In [ ]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.14495877038053528, 'rouge2': 0.025132885690161234, 'rougeL': 0.10734854191439931, 'rougeLsum': 0.10598143808643898}
INSTRUCT MODEL:
{'rouge1': 0.16403334487739019, 'rouge2': 0.05360793615254001, 'rougeL': 0.13371414071699528, 'rougeLsum': 0.13496962262442147}
PEFT MODEL:
{'rouge1': 0.11845824916588851, 'rouge2': 0.027788553320893507, 'rougeL': 0.08328314146211482, 'rougeLsum': 0.08144067107488592}


In [ ]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: -2.65%
rouge2: 0.27%
rougeL: -2.41%
rougeLsum: -2.45%


In [ ]:
print("Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(instruct_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL
rouge1: -4.56%
rouge2: -2.58%
rougeL: -5.04%
rougeLsum: -5.35%


## The Second attempt


In [ ]:
# python
!pip install pytesseract transformers datasets rouge-score nltk tensorboard py7zr --upgrade
! pip install transformers[torch]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 19.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to dete

In [ ]:
# install git-fls for pushing model and logs to the hugging face hub
!sudo apt-get install git-lfs --yes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
dataset_id = "samsum"


In [ ]:
from datasets import load_dataset

# Load dataset from the hub
dataset = load_dataset(dataset_id)

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

# Train dataset size: 14732
# Test dataset size: 819

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Train dataset size: 14732
Test dataset size: 819


In [ ]:
from random import randrange


sample = dataset['train'][randrange(len(dataset["train"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")
print(f"summary: \n{sample['summary']}\n---------------")

dialogue: 
Kirsten: yay! got tickets! 
Viola: wot tickets luv?
Kirsten: Ed's gig!
Viola: wow. u mean ed sheeran. Lucky!
Kirsten: I know, so excited!
Viola: when's that?
Kirsten: in august. still plenty of time but can't wait already
Viola: I know, if you're fan it's always like that
Kirsten: yep. I luv this guy.
---------------
summary: 
Kristen got tickets for Ed Sheeran's gig in August.
---------------


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/15551 [00:00<?, ? examples/s]

Max source length: 512


Map:   0%|          | 0/15551 [00:00<?, ? examples/s]

Max target length: 95


In [ ]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["summarize: " + item for item in sample["dialogue"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [ ]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id="google/flan-t5-base"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
# peft
# هسه حنبدي نشتغل ب لورا و الي تجمد الويتس مال مودل و تجيب ابعاد اصغرr
# و تدربهم ع بيانات الفاين تيون و بعدها ترجع و تدمجهن بالمودل الجديد

from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    # و هذا الدايمنشن الي كلنه يكون بين ال8 و 32
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    # هذا نوع التاسك
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [ ]:
# هنا انطيته المودل مالتي حته يجمده و انطيته اعدادات لورا حته يضبطها
peft_model = get_peft_model(model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

NameError: name 'print_number_of_trainable_model_parameters' is not defined

In [ ]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-{dataset_id}-moh"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir='/content/drive/MyDrive/Graduation_Models/Summary_Model/peftmodel/output',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    # logging & evaluation strategies
    logging_dir="/content/drive/MyDrive/Graduation_Models/Summary_Model/peftmodel/log",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    # push_to_hub=False,
    hub_strategy="every_save",
    # hub_model_id=repository_id,
    # hub_token=HfFolder.get_token(),
)

# Create Trainer instance
peft_trainer = Seq2SeqTrainer(
    model=peft_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
peft_trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.455900,1.412373,46.246600,22.113900,38.553400,42.569700,17.195360


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


RuntimeError: 
            Some tensors share memory, this will lead to duplicate memory on disk and potential differences when loading them again: [{'base_model.model.encoder.embed_tokens.weight', 'base_model.model.shared.weight', 'base_model.model.decoder.embed_tokens.weight'}].
            A potential way to correctly save your model is to use `save_model`.
            More information at https://huggingface.co/docs/safetensors/torch_shared_tensors
            

In [ ]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-{dataset_id}-moh"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir='/content/drive/MyDrive/Graduation_Models/Summary_Model/output',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    # logging & evaluation strategies
    logging_dir="/content/drive/MyDrive/Graduation_Models/Summary_Model/Log",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    # push_to_hub=False,
    hub_strategy="every_save",
    # hub_model_id=repository_id,
    # hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
# Start training
# epochs
#5	1.196000	1.374168	47.410900	23.618600	39.638300	43.556700	17.313797
trainer.train(resume_from_checkpoint=True)

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
5,1.196000,1.374168,47.410900,23.618600,39.638300,43.556700,17.313797


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=9210, training_loss=0.24027443056386147, metrics={'train_runtime': 2426.5374, 'train_samples_per_second': 30.356, 'train_steps_per_second': 3.796, 'total_flos': 5.048305105187635e+16, 'train_loss': 0.24027443056386147, 'epoch': 5.0})

In [ ]:
trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 1.3700677156448364,
 'eval_rouge1': 47.4992,
 'eval_rouge2': 23.573,
 'eval_rougeL': 39.8929,
 'eval_rougeLsum': 43.647,
 'eval_gen_len': 17.35286935286935,
 'eval_runtime': 123.4031,
 'eval_samples_per_second': 6.637,
 'eval_steps_per_second': 0.835,
 'epoch': 5.0}

In [ ]:
# Test
from transformers import pipeline
from random import randrange

# load model and tokenizer from huggingface hub with pipeline
summarizer = pipeline("summarization", model="mohammedRiad/flant5-summaryModel", device=0)

# select a random test sample
sample = dataset['test'][randrange(len(dataset["test"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")

# summarize dialogue
res = summarizer(sample["dialogue"])

print(f"flan-t5-base summary:\n{res[0]['summary_text']}")

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

dialogue: 
Jill: so how was your date anyway? :)
Susan: it was perfect, he was so sweet!! <3
Jill: tell me everything!!
Susan: so first he picked me up from home, all dressed up and everything
Jill: suit on? :D
Susan: nooo, not like that, but he had a really nice black shirt, elegant shoes, looked like Brad Pitt hahaha
Jill: hahahahaha
Susan: <file_gif>
Jill: so where did he take you?
Susan: that's the best part! We went to rollerskating disco!!!
Jill: rollerskating what? are you serious? :/
Susan: no no, listen - it was so awesome, I am tired of all this dull restaurant dates and stuff, it was actually something original
Jill: if you say so...
Susan: you are just jealous Jill :D
Jill: I am not!!!!!!!
Susan: yes you are :*
---------------
flan-t5-base summary:
Susan's date was perfect. He picked her up from home and took her to rollerskating disco. She is tired of dull restaurant dates.
